In [68]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_dynamics_upper.jl")
# include("./funlopt/funl_constraint.jl")
# include("./trajopt/scaling.jl")

LoadError: LoadError: InterruptException:
in expression starting at /Users/taewankim/Documents/Code/Personal/JointSynthesisFreeFinal/trajopt/dynamics.jl:90
in expression starting at /Users/taewankim/Documents/Code/Personal/JointSynthesisFreeFinal/funlopt/funl_utils.jl:2
in expression starting at /Users/taewankim/Documents/Code/Personal/JointSynthesisFreeFinal/funlopt/funl_dynamics_upper.jl:3

In [69]:
# # load nominal trajectory
# using JLD2, FileIO
# @load "./data/quadstar_N15_traj" my_dict
# xnom = my_dict["x"]
# unom = my_dict["u"]
# tnom = my_dict["t"];
# N = size(xnom,2) - 1
# dtnom = zeros(N)
# for i in 1:N
#     dtnom[i] = tnom[i+1]-tnom[i]
# end
# @load "./data/quadstar_N15_funl" my_dict
# Qnom = my_dict["Q"]
# Knom = my_dict["K"]
# Znom = my_dict["Z"];


@load "./data/xuQYZ_unicycle_0328" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"]
Qnom = my_dict["Q"]
Ynom = my_dict["Y"]
Znom = my_dict["Z"]
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end


In [70]:
# dynamics = QuadrotorDynamics()
dynamics = Unicycle()
ix = dynamics.ix
iu = dynamics.iu
decay_rate = 0.1
DLMI = NonlinearDLMI(decay_rate,ix,iu,dynamics.Cv,dynamics.Dvu)
is = DLMI.is

1

In [71]:
idx = 4
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])

([0.0 0.0 -0.8563473835516268; 0.0 0.0 1.2805836524424632; 0.0 0.0 0.0], [0.831263029672674 0.0; 0.5558792814086592 0.0; 0.0 1.0])

In [83]:
include("./funlopt/funl_dynamics_upper.jl")

diff_numeric (generic function with 1 method)

In [84]:
idx = 4
Q = Qnom[:,:,idx]
# K = Knom[:,:,idx]
K = Ynom[:,:,idx] * inv(Qnom[:,:,idx])
Z = Znom[:,:,idx]
lam = 1.5
X = vec_upper(Q,ix)
U = vcat(vec(K),vec_upper(Z,ix),lam)
;

# forward

In [85]:
L = (A+B*K)*Q + 0.5*DLMI.alpha*Q
M = (DLMI.C+DLMI.D*K)*Q
dQ = L + L' + lam*M'*M + Z
;

In [86]:
dq_upper = forward(DLMI,X,U,A,B)
dQ_ = inv_vec_upper(dq_upper,ix)
;

In [87]:
sum(abs.(dq_upper - vec_upper(dQ,ix))) / sum(abs.(dq_upper))

1.1565947809466846e-16

In [88]:
sum(abs.(dQ-dQ_)) / sum(abs.(dQ))

1.2803897627623105e-16

# Jacobian

In [89]:
Fx_n,Fu_n = diff_numeric(DLMI,X,U,A,B)

([-0.015112217050045729 -0.20892163971439004 … 0.0 0.0; -0.03232918644789606 -0.016231605783104897 … 4.386538234422915 0.0; … ; 0.0 0.8202561646467075 … 3.8485932496841997 -1.8980984996305779; 0.0 0.0 … -2.7010434311814606 7.714537493418902], [2.3109952551312745 0.0 … 0.0 12.36987409973517; 0.2362610874697566 0.0 … 0.0 -7.297776383114979; … ; 0.2723916245158762 -1.6872438208665699 … 0.0 -8.119554763659835; 0.8614602955058217 6.990545892622322 … 1.0 14.605011752806604])

In [90]:
Fx,Fu = diff(DLMI,X,U,A,B)

9×9 BlockDiagonal{Float64, Matrix{Float64}}:
 -0.00755611  -0.104461    4.38654  …   0.0          0.0         0.0
 -0.0323292   -0.0086755  -1.8981       0.0          0.0         0.0
  0.820256    -1.35052     3.85727      0.0          0.0         0.0
  0.0          0.0         0.0          0.0          0.0         0.0
  0.0          0.0         0.0          0.0          0.0         0.0
  0.0          0.0         0.0      …   0.0          0.0         0.0
  0.0          0.0         0.0         -0.00755611  -0.104461    4.38654
  0.0          0.0         0.0         -0.0323292   -0.0086755  -1.8981
  0.0          0.0         0.0          0.820256    -1.35052     3.85727

([-0.015112217074851664 -0.20892164001222335 … 0.0 0.0; -0.03232918624076592 -0.01623160607640084 … 4.386538234467287 0.0; … ; 0.0 0.8202561644394846 … 3.848593249517348 -1.8980984993850532; 0.0 0.0 … -2.7010434304852047 7.714537494112646], [2.310995255348861 0.0 … 0.0 12.369874099894696; 0.2362610875356046 0.0 … 0.0 -7.297776383019955; … ; 0.272391624538205 -1.687243820790398 … 0.0 -8.119554763808242; 0.8614602951082494 6.990545892963479 … 1.0 14.605011753039244])

In [91]:
sum(abs.(Fx_n-Fx)) / sum(abs.(Fx_n))

1.1228474894848514e-10

In [92]:
sum(abs.(Fu_n-Fu)) / sum(abs.(Fu_n))

4.441293854309866e-11

In [ ]:
Fx_n - Fx

In [ ]:
Fx

In [ ]:
Fu - Fu_n

In [ ]:
sum(abs.((Fu - Fu_n) .> 1e-8))

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark diff_numeric(DLMI,X,U,A,B)

In [ ]:
@benchmark diff(DLMI,X,U,A,B)

In [ ]:
Fu_n

In [ ]:
Fu

In [ ]:
DLMI.D

In [ ]:
Fu_n

In [ ]:
Fu

In [ ]:
print_jl(Fx)
print_jl(Fu)

# constraint

In [ ]:
idx = 4
# Q = Qnom[:,:,idx]
# K = Ynom[:,:,idx] * inv(Qnom[:,:,idx])
# Z = Znom[:,:,idx]
# lam = 1.3
Q = randn(3,3)
K = randn(2,3)
Z = randn(3,3)
lam = 1.5
X = vec(Q)
U = vcat(vec(K),vec(Z),lam)

In [ ]:
a = [1.5;2.0]
function evaluate(q,k)
    Q = reshape(q,(3,3))
    K = reshape(k,(2,3))
    return a'*K*Q*K'*a
end

In [ ]:
evaluate(vec(Q),vec(K))

In [ ]:
function eval_diff_numeric(q,k)
    ix = length(q)
    iu = length(k)

    eps_x = Diagonal{Float64}(I, ix)
    eps_u = Diagonal{Float64}(I, iu)
    fx = zeros(1,ix)
    fu = zeros(1,iu)

    h = 2^(-18)
    for i in 1:ix
        fx[:,i] .= (evaluate(q+h*eps_x[:,i],k) - evaluate(q-h*eps_x[:,i],k)) / (2*h)
    end
    for i in 1:iu
        fu[:,i] .= (evaluate(q,k+h*eps_u[:,i]) - evaluate(q,k-h*eps_u[:,i])) / (2*h)
    end
    return fx,fu
end

In [ ]:
Cm = DLMI.Cm

In [ ]:
fk = vec(K)' * kron(Q, a*a') + (kron(Q, a*a') * vec(K))'
# fk = 2 * vec(K)' * kron(Q, a*a')
fq = kron(a'*K,a'*K)

In [ ]:
fq_,fk_ = eval_diff_numeric(vec(Q),vec(K))

In [ ]:
println(sum(abs.(fq-fq_)))
println(sum(abs.(fk-fk_)))

In [ ]:
print_jl(fq_)
print_jl(fk_)